<a href="https://colab.research.google.com/github/aluposto/video_codec_test_11/blob/main/dcvc_Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch, os, platform
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi
!python --version


CUDA available: True
Sun Aug 31 12:40:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [ ]:
!rm -rf DCVC
!git clone https://github.com/microsoft/DCVC.git
%cd DCVC

# Install build tools
!sudo apt-get update -y
!sudo apt-get install -y cmake g++ ninja-build

# Install PyTorch 2.6 with CUDA 12.6 (fallback to CUDA 12.1 if needed)
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 || \
 pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Python dependencies
!pip install -r requirements.txt

# Build C++ bitstream writer
%cd src/cpp
!pip install .
# Build CUDA fused inference kernels
%cd ../layers/extensions/inference
!pip install .

# Back to root
%cd ../../../..


Cloning into 'DCVC'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 453 (delta 76), reused 45 (delta 45), pack-reused 324 (from 2)
Receiving objects: 100% (453/453), 7.43 MiB | 13.37 MiB/s, done.
Resolving deltas: 100% (184/184), done.
/content/DCVC
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcon

In [ ]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Torch: 2.8.0+cu126
CUDA available: True
GPU name: Tesla T4


In [ ]:
!pip install tqdm requests


In [ ]:
import requests
from tqdm import tqdm

url = "https://ultravideo.fi/video/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"
output_path = "/media/data/HEVC_B/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z"

# Ensure folder exists
import os
os.makedirs("/media/data/HEVC_B", exist_ok=True)

# Stream download with progress bar
response = requests.get(url, stream=True)
total_size = int(response.headers.get("content-length", 0))
block_size = 1024 * 1024  # 1 MB chunks
t = tqdm(total=total_size, unit="B", unit_scale=True, desc="Downloading")

with open(output_path, "wb") as f:
    for data in response.iter_content(block_size):
        f.write(data)
        t.update(len(data))
t.close()

print("✅ Download finished:", output_path)


Downloading: 100%|██████████| 925M/925M [00:44<00:00, 20.6MB/s]

✅ Download finished: /media/data/HEVC_B/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z


In [ ]:
!sudo apt-get install -y p7zip-full
!7z x /media/data/HEVC_B/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z -o/media/data/HEVC_B/
!ls -lh /media/data/HEVC_B


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /media/data/HEVC_B/                             1 file, 925430047 bytes (883 MiB)

Extracting archive: /media/data/HEVC_B/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
--
Path = /media/data/HEVC_B/Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
Type = 7z
Physical Size = 925430047
Headers Size = 246
Method = LZMA:16 BZip2
Solid = -
Blocks = 2

  0%      0% - Beauty_1920x1080_120fps_420_8bit_YUV.yuv                                  

In [ ]:
!ls -lh /media/data/HEVC_B


total 2.6G
-rw-r--r-- 1 root root 883M Aug 31 13:17 Beauty_1920x1080_120fps_420_8bit_YUV_RAW.7z
-rw-r--r-- 1 root root 1.8G Oct  7  2013 Beauty_1920x1080_120fps_420_8bit_YUV.yuv
-rw-r--r-- 1 root root  249 Oct 24  2013 Beauty_Copyright.txt


In [ ]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./dataset_config_example_yuv420.json"  # expects /media/data/HEVC_B
OUTJSON="output.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 4 \
  --test_config "$CONFIG" \
  --cuda 1 \
  -w 1 \
  --write_stream 1 \
  --force_zero_thres 0.12 \
  --output_path "$OUTJSON" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num -1 \
  --check_existing 0 \
  --verbose 1


/content/DCVC
testing 4 rates, using qp: 0, 21, 42, 63, 
  0% 0/212 [00:00<?, ?it/s]created folder: out_bin/UVG
  0% 0/212 [00:05<?, ?it/s]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.12/concurrent/futures/process.py", line 264, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 370, in worker
    result = run_one_point_with_stream(p_frame_net, i_frame_net, args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 147, in run_one_point_with_stream
    src_reader = get_src_reader(args)
                 ^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 70, in get_src_reader
    src_reader = YUV420Reader(args['src_path'], args['src_width'], args['src_height'])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
!mkdir -p /media/data/UVG
!mv /media/data/HEVC_B/Beauty_1920x1080_120fps_420_8bit_YUV.yuv /media/data/UVG/
!ls -lh /media/data/UVG


total 1.8G
-rw-r--r-- 1 root root 1.8G Oct  7  2013 Beauty_1920x1080_120fps_420_8bit_YUV.yuv


In [ ]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./dataset_config_example_yuv420.json"  # expects /media/data/HEVC_B
OUTJSON="output.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 4 \
  --test_config "$CONFIG" \
  --cuda 1 \
  -w 1 \
  --write_stream 1 \
  --force_zero_thres 0.12 \
  --output_path "$OUTJSON" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num -1 \
  --check_existing 0 \
  --verbose 1


/content/DCVC
testing 4 rates, using qp: 0, 21, 42, 63, 
  0% 0/212 [00:00<?, ?it/s]encoding/decoding 600 frames, average encoding time 44.423 ms, average decoding time 51.917 ms.
  0% 1/212 [02:41<9:28:27, 161.64s/it]Exception ignored in: <generator object tqdm.__iter__ at 0x7fc25a884940>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 1275, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 686, in _decr_instances
    @classmethod

KeyboardInterrupt: 
Traceback (most recent call last):
  File "/content/DCVC/test_video.py", line 541, in <module>
    main()
  File "/content/DCVC/test_video.py", line 514, in main
    result = obj.result()
             ^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 451, in result
    self._condition.wait(timeout)
  Fi

KeyboardInterrupt: 

In [ ]:
import json

config_path = "/content/DCVC/my_beauty_config.json"

beauty_config = {
    "root_path": "/media/data",
    "test_classes": {
        "UVG": {
            "root": "UVG",
            "base_path": "UVG",   # ✅ add base_path
            "test": 1,
            "src_type": "yuv",
            "sequences": {
                "Beauty": {
                    "path": "Beauty_1920x1080_120fps_420_8bit_YUV.yuv",
                    "width": 1920,
                    "height": 1080,
                    "fps": 120,
                    "frame_num": -1,
                    "frames": -1,
                    "intra_period": -1
                }
            }
        }
    }
}

with open(config_path, "w") as f:
    json.dump(beauty_config, f, indent=2)

print("✅ Config with base_path added:", config_path)


✅ Config with base_path added: /content/DCVC/my_beauty_config.json


In [ ]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./my_beauty_config.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 1 \
  --qp_i 42 \
  --qp_p 42 \
  --test_config "$CONFIG" \
  --cuda 1 \
  --worker 1 \
  --write_stream 1 \
  --output_path "output_short.json" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num 100 \
  --check_existing 0 \
  --verbose 1


/content/DCVC
testing 1 rates, using qp: 42, 
  0% 0/1 [00:05<?, ?it/s]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.12/concurrent/futures/process.py", line 264, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 370, in worker
    result = run_one_point_with_stream(p_frame_net, i_frame_net, args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 147, in run_one_point_with_stream
    src_reader = get_src_reader(args)
                 ^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 71, in get_src_reader
    return src_reader
           ^^^^^^^^^^
UnboundLocalError: cannot access local variable 'src_reader' where it is not associated with a value
"""

The above exception was the direct cause of the following exception:



In [ ]:
import json

config_path = "/content/DCVC/my_beauty_config.json"

beauty_config = {
    "root_path": "/content/DCVC",   # ✅ fix for Colab
    "test_classes": {
        "UVG": {
            "root": "UVG",
            "base_path": "UVG",
            "test": 1,
            "src_type": "yuv",
            "sequences": {
                "Beauty": {
                    "path": "Beauty_1920x1080_120fps_420_8bit_YUV.yuv",
                    "width": 1920,
                    "height": 1080,
                    "fps": 120,
                    "frame_num": -1,
                    "frames": -1,
                    "intra_period": -1
                }
            }
        }
    }
}

with open(config_path, "w") as f:
    json.dump(beauty_config, f, indent=2)

print("✅ Colab path fixed:", config_path)


✅ Colab path fixed: /content/DCVC/my_beauty_config.json


In [ ]:
%cd /content/DCVC

MODEL_I="./checkpoints/cvpr2025_image.pth.tar"
MODEL_P="./checkpoints/cvpr2025_video.pth.tar"
CONFIG="./my_beauty_config.json"

!python test_video.py \
  --model_path_i "$MODEL_I" \
  --model_path_p "$MODEL_P" \
  --rate_num 1 \
  --qp_i 42 \
  --qp_p 42 \
  --test_config "$CONFIG" \
  --cuda 1 \
  --worker 1 \
  --write_stream 1 \
  --output_path "output_short.json" \
  --force_intra_period -1 \
  --reset_interval 64 \
  --force_frame_num 100 \
  --check_existing 0 \
  --verbose 1


/content/DCVC
testing 1 rates, using qp: 42, 
  0% 0/1 [00:05<?, ?it/s]
concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.12/concurrent/futures/process.py", line 264, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 370, in worker
    result = run_one_point_with_stream(p_frame_net, i_frame_net, args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 147, in run_one_point_with_stream
    src_reader = get_src_reader(args)
                 ^^^^^^^^^^^^^^^^^^^^
  File "/content/DCVC/test_video.py", line 71, in get_src_reader
    return src_reader
           ^^^^^^^^^^
UnboundLocalError: cannot access local variable 'src_reader' where it is not associated with a value
"""

The above exception was the direct cause of the following exception:



In [ ]:
import json
with open("/content/DCVC/my_beauty_config.json") as f:
    cfg = json.load(f)

print("src_type:", cfg["test_classes"]["UVG"]["src_type"])
print("seq path:", cfg["test_classes"]["UVG"]["sequences"]["Beauty"]["path"])


src_type: yuv
seq path: Beauty_1920x1080_120fps_420_8bit_YUV.yuv


In [ ]:
import os
print(os.path.exists("/content/DCVC/UVG/Beauty_1920x1080_120fps_420_8bit_YUV.yuv"))


False
